# Getting started

## One time setup 

You should have cloned the `aeon_mecha_de` folder as follows:

```
cd 
mkdir repos
cd repos
git clone https://gitlab.com/sainsbury-wellcome-centre/delab/arena/aeon_mecha_de.git
```

You should also have [installed Miniconda](https://docs.conda.io/en/latest/miniconda.html) or Anaconda.

You know this is successful if you can run `conda env list` without any errors and see a list of environments.

It is highly recommended to install [mamba](https://mamba.readthedocs.io/en/latest/) as follows:

`conda install mamba -n base -c conda-forge`

Then, you can set up your development / analysis envirment as follows:

```
cd ~/repos/aeon_mecha_de
mamba env create -f env.yml
```

Once this is all finished you should be ready to go!

### Adding aeon to the path

Each time you want to work on the project you need to activate your environment:

`conda activate aeon_env`

But this doesn't add the `aeon` code itself to the path. You can do it in a few ways, but the following code block does it for you.

In [1]:
import sys
from os.path import expanduser
# As long as you have cloned the aeon_mecha_de folder into 
# repos in your home filter
sys.path.append(expanduser('~/repos/aeon_mecha_de'))

# This path is only true if you are on pop.erlichlab.org
dataroot = '/var/www/html/delab/data/arena0.1/socialexperiment0/'
figpath = '/var/www/html/delab/figures/'

In [2]:
# import aeon.util.helpers as helpers
import numpy as np
import pandas as pd
import aeon.analyze.patches as patches
import os 
import aeon.preprocess.api as api
import matplotlib.pyplot as plt

In [3]:
%reload_ext autoreload
%autoreload 1
%aimport aeon.util.helpers 
%aimport aeon.util.plotting 
helpers = aeon.util.helpers
plotting = aeon.util.plotting

In [4]:
sessdf = api.sessiondata(dataroot)
sessdf = api.sessionduration(sessdf)                                     # compute session duration
sessdf = sessdf[~sessdf.id.str.contains('test')]
sessdf = sessdf[~sessdf.id.str.contains('jeff')]
sessdf = sessdf[~sessdf.id.str.contains('OAA')]
sessdf = sessdf[~sessdf.id.str.contains('rew')]
sessdf = sessdf[~sessdf.id.str.contains('Animal')]

sessdf.reset_index(inplace=True, drop=True)

In [63]:
df = sessdf.copy()
#helpers.mergeSocial(df)
#helpers.merge(df)


In [19]:
pd.set_option('display.max_rows', 500) 

In [61]:
valid_id_file = expanduser('~/mnt/delab/conf/valid_ids.csv')
vdf = pd.read_csv(valid_id_file)
valid_ids = list(vdf.id.values[vdf.real.values==1])

In [62]:
valid_ids

['BAA-1100704', 'BAA-1100705', 'BAA-1100706']

In [64]:
df.id = df.id.apply(helpers.fixID, valid_ids=valid_ids)

In [66]:
helpers.mergeSocial(df)
helpers.merge(df)

In [68]:
sessdf.loc[:,['id','start']]

,id,start
0,BAA-1100704,2021-11-30 15:09:05.403999805
1,BAA-1100705,2021-11-30 16:12:03.540540218
2,BAA-1100706,2021-11-30 17:15:39.932059765
3,BAA-1100704,2021-12-01 14:11:52.115520000
4,BAA-1100705,2021-12-01 15:20:27.967520237
5,BAA-1100706,2021-12-01 17:11:50.997859955
6,BAA-1100704,2021-12-02 11:16:35.048799992
7,BAA-1100705,2021-12-02 13:25:47.708580017
8,BAA-1100706,2021-12-02 15:42:01.764289856
9,BAA-1100704,2021-12-03 12:13:45.891069889


In [14]:
np.where([True, False, True])[0]

array([0, 2])

In [30]:
df.iloc[social[0]+0]

id                   BAA-1100704; BAA-1100705
weight_start                             24.2
weight_end                               24.2
start           2021-12-07 11:19:42.800479889
end             2021-12-07 11:19:56.736480236
duration            0 days 00:00:13.936000347
Name: 17, dtype: object

## Fix the bad data

In [ ]:
df = sessdf.copy()
helpers.merge(df)
helpers.merge(df,first=[15])
helpers.merge(df,first=[32,35], merge_id=True)
helpers.markSessionEnded(df)
#df.loc[4,'end'] = df.loc[4,'start'] + pd.DateOffset(minutes=100)
#df.loc[[19,32,34],'end'] = df.loc[[19,32,34],'start'] + pd.DateOffset(minutes=150)

#df.duration = df.end - df.start
# df 

In [ ]:
session_list = df.itertuples()

In [ ]:
session = next(session_list)
session

In [ ]:
for session in session_list:
    try:
        meta = {'session':session}
        filename = plotting.plotFileName(os.path.join(figpath,'patch'), 'patch', meta)
        if not os.path.exists(filename):
            data = helpers.getWheelData(dataroot, session.start, session.end)
            data['meta'] = meta
            data['filename'] = filename
            fig = plotting.plotWheelData(**data)
        else:
            print(f'{filename} exists. Skipping.')
    except IndexError as e:
        print(e)

In [ ]:
start, end, sdf = helpers.splitOnStateChange(dataroot, session.start, session.end)
start, end, sdf

In [ ]:
posdata = helpers.getPositionData(dataroot, start,end)

In [ ]:
before = helpers.getPositionData(dataroot, start[0], end[0])
after = helpers.getPositionData(dataroot,start[1], start[1] + pd.DateOffset(minutes=30) )

In [ ]:
before['title_str'] = f'{session.id}, {start[0]:%m/%d %H:%M}-{end[0]:%H:%M}'
fig, ax = plotting.positionmap(**before, bins=200,vmin=.01,vmax=10)
fig.patch.set_facecolor('white')
fig.savefig(f"/www-delab/figures/posmap/{session.id.replace(';','')}_{start[0]:%m%d}_pos_before.png", dpi=400)



In [ ]:
session.id.replace(';','')

In [ ]:
after['title_str'] = f'{session.id}, {start[1]:%m/%d %H:%M}-{start[1] + pd.DateOffset(minutes=30):%H:%M}'
fig, ax = plotting.positionmap(**after, bins=200,vmin=.01,vmax=10)
fig.patch.set_facecolor('white')
fig.savefig(f"/www-delab/figures/posmap/{session.id.replace(';','')}_{start[0]:%m%d}_pos_after.png", dpi=400)
